# Kaggle MNIST Challenge

Using a Deep Convolutional Neural Network to classify digits for the Kaggle MNIST challenge.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Add, Dense, Dropout
from keras.layers import Flatten, AveragePooling2D, MaxPooling2D, Conv2D, Activation, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, LearningRateScheduler, ModelCheckpoint, EarlyStopping

from keras_contrib.layers.advanced_activations import SineReLU

from sklearn.model_selection import train_test_split

import pandas as pd

import os

import matplotlib.pyplot as plt
%matplotlib inline

#### Load data

In [ ]:
n_classes = 10

raw_data = np.loadtxt('kaggle/datasets/mnist/train.csv', skiprows=1, dtype='int', delimiter=',')
X_train, X_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size = 0.2)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_val = X_val.astype("float32")/255.
X_test = X_test.astype("float32")/255.


y_train = keras.utils.to_categorical(y_train, n_classes)
y_val = keras.utils.to_categorical(y_val, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)


In [ ]:
output_dir = 'model_output/multi-conv'

#### Design Neural Network architecture

In [ ]:
epsilon = 0.0025

model = Sequential()

model.add(Conv2D(28, 7, padding = 'same', input_shape = (28, 28, 1)))
model.add(SineReLU(epsilon))
model.add(Conv2D(28, 7, padding = 'same'))
model.add(SineReLU(epsilon))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, 3, padding = 'same'))
model.add(SineReLU(epsilon))
model.add(Conv2D(32, 3, padding = 'same'))
model.add(SineReLU(epsilon))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(44, 2, padding = 'same'))
model.add(SineReLU(epsilon))
model.add(Conv2D(44, 2, padding = 'same'))
model.add(SineReLU(epsilon))
model.add(Conv2D(48, 2))
model.add(SineReLU(epsilon))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(SineReLU(epsilon))
model.add(Dropout(0.25))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.summary()

#### Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

test_datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

train_generator = train_datagen.flow(X_train, y_train, batch_size = 64)

validation_generator = test_datagen.flow(X_val, y_val, batch_size = 64)

#### Learning Rate Scheduler

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

#### Save the Best

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath=output_dir+'/weights-cnn-mnist.hdf5', save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=3)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### TensorBoard

In [ ]:
tensorboard = TensorBoard("../logs/cnn-mnist-ReLUs-30-epochs-EkhoNet8-data-augmentation")

#### Train!

In [ ]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch = 3500,
                           epochs = 15,
                           verbose = 1,
                           validation_data = validation_generator,
                           validation_steps = 3500,
                           callbacks = [modelCheckpoint, earlyStopping])#, annealer, tensorboard])

# hist = model.fit(X_train, y_train, batch_size = 32,
#                            epochs = 40,
#                            verbose = 1,
#                            validation_split = 0.1,
#                            callbacks=[annealer, tensorboard])

#### Load Model

In [ ]:
# Zero indexed -> get the epoch with highest accuracy.
saved_model = keras.models.load_model(output_dir + '/weights-cnn-mnist.hdf5')

#### Test Final Accuracy

In [ ]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

#### Save Submission

In [ ]:
X_test_sub = np.loadtxt('kaggle/datasets/mnist/test.csv', skiprows=1, dtype='int', delimiter=',')
X_test_sub = X_test_sub.reshape(28000, 28, 28, 1).astype('float32') / 255.

predictions = model.predict(X_test_sub, verbose = 2)
predictions = np.argmax(predictions, axis = 1)

pd.DataFrame({"ImageId": list(range(1, len(predictions) + 1)), "Label": predictions}).to_csv('kaggle/results/mnist/submission-ReLUs-30-epochs-EkhoNet8-data-augmentation.csv', index = False, header = True)